In [74]:
%pip install langchain
%pip install langgraph
%pip install weaviate-client
%pip install ipywidgets
%pip install tqdm
%pip install -U langchain-ollama
%pip install -U langchain-openai
%pip install -U langchain-openai
%pip install -U langchain-weaviate



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=8494) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [25]:
import weaviate
import weaviate.classes.config as wcd
from langchain_weaviate import WeaviateVectorStore
from langchain_openai import AzureOpenAIEmbeddings
import os

client = weaviate.connect_to_local()
collection_name = "products"


api_key = os.environ.get("AZURE_OPENAI_KEY")
azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
api_version = "2024-10-21"

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-ada-003",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)


weaviate_client = weaviate.connect_to_local()
db = WeaviateVectorStore(client=weaviate_client, embedding=embeddings, index_name=collection_name, text_key="description")



/Users/gaspar_d/Projects/weaviate-playbooks/.venv/lib/python3.12/site-packages/weaviate/warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


In [3]:
ollama_api_endpoint = "http://host.docker.internal:11434"

if (client.collections.exists(collection_name)):
    client.collections.delete(collection_name)

client.collections.create(name=collection_name,
                          description="Contains all product for ecommerce website",
                          vectorizer_config=wcd.Configure.Vectorizer.none(),
                          properties=[
                              wcd.Property(name="title", data_type=wcd.DataType.TEXT),
                              wcd.Property(name="category", data_type=wcd.DataType.TEXT),
                              wcd.Property(name="description", data_type=wcd.DataType.TEXT),
                              wcd.Property(name="price", data_type=wcd.DataType.NUMBER, skip_vectorization=True),
                          ]
                          )

In [4]:
collection = client.collections.get(collection_name)

In [5]:
import tqdm.notebook as tqdm
from datasets import load_dataset



# List of available datasets available on:
# https://amazon-reviews-2023.github.io/index.html
dataset_names = ["raw_meta_Clothing_Shoes_and_Jewelry", "raw_meta_Toys_and_Games", "raw_meta_All_Beauty"]

for dataset_name in dataset_names:
    dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", dataset_name, split="full", trust_remote_code=True, streaming=True)
    texts = []
    metadatas = []
    for item in tqdm.tqdm(iter(dataset)):
        dict = {
            "title": item["title"],
            "category": item["main_category"],
            "price": -1 if item["price"] == "None" else float(item["price"]) ,
        }
        texts.append("\n".join(item["description"]))
        metadatas.append(dict)
    db.add_texts(texts=texts, metadatas=metadatas, )

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [7]:
from langchain_core.tools import tool
from weaviate.classes.query import Filter

@tool(name_or_callable="weaviate_search")
def weaviate_search(query: str) -> str:
    """search for products in database

    Args:
        query: query to search for
    """
    response = collection.query.near_text(query, limit=1)
    if len(response) == 0:
        return "No product found"
    return response.objects[0]


@tool(name_or_callable="weaviate_search_with_price_filter")
def weaviate_search_with_price_filter(query: str, maximum_price: int = 9999, minimum_price = 0) -> str:
    """search for products in database and filter results by price

    Args:
        query: query to search for
        maximum_price: maximum price to search for, if not specified, defaults to 9999
        minimum_price: minimum price to search for, if not specified, defaults to 0
    """

    response = collection.query.near_text(query, limit=1, filters=Filter.all_of([
        Filter.by_property("price").greater_than(minimum_price),
        Filter.by_property("price").less_than(maximum_price),
    ]))
    if len(response) == 0:
        return "No product found"
    return response.objects[0]




In [106]:
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
import os

os.environ["AZURE_OPENAI_ENDPOINT"] = os.environ.get("AZURE_OPENAI_COGNITIVE_ENDPOINT")
os.environ["AZURE_OPENAI_API_KEY"] = os.environ.get("AZURE_OPENAI_COGNITIVE_API_KEY")

model = AzureChatOpenAI(
    openai_api_type="azure",
    azure_deployment="GPT",
    api_version="2024-10-21",
    temperature=0,
)

tools = [weaviate_search, weaviate_search_with_price_filter]
model_with_tools = model.bind_tools(tools)

In [107]:
response = model_with_tools.invoke([HumanMessage(content="Blue lipstick")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'weaviate_search', 'args': {'query': 'blue lipstick'}, 'id': 'call_5M0zQCLMYYVIqGwFGbatbw3V', 'type': 'tool_call'}]


In [109]:
response = model_with_tools.invoke([HumanMessage(content="Blue lipstick below 10$")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")


ContentString: 
ToolCalls: [{'name': 'weaviate_search_with_price_filter', 'args': {'query': 'blue lipstick', 'maximum_price': 10}, 'id': 'call_sNlTHHdS0HeSpjvwljVlpM62', 'type': 'tool_call'}]


In [108]:
from pydantic import BaseModel, Field
from typing import Literal, TypedDict, List
from langchain_core.messages import HumanMessage, SystemMessage, BaseMessage
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

tools = [weaviate_search, weaviate_search_with_price_filter]
tool_node = ToolNode(tools)

class Route(BaseModel):
    step: Literal["synthetizer", "chat"] = Field(
        None, description="The next step in the routing process"
    )

# State
class State(TypedDict):
    messages: List[BaseMessage]
    input: str
    decision: str
    output: str

router = model.with_structured_output(Route)

def should_continue(state: State) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

def tools_callback(state: State) -> Literal["synthetizer", "chat"]:
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

def llm_call_router(state: State):
    decision = router.invoke(
        [
            SystemMessage(
                content="""You are an expert in product, if the user is looking for a specific product, redirect it to 'synthetizer'.
                If you are unsure of what he is looking for, rediret it to 'chat'
                """
            ),
            HumanMessage(content=state["messages"][0].content),
        ]
    )

    return {"decision": decision.step }


# Define the function that calls the model
def call_model_synthetize(state: State):
    """Provide ONLY a list of result separated by line"""
    input_message = state['messages'][0].content
    response = model.invoke([
        SystemMessage(
            content="Generate an output containing only a bullet point list of products without any description",
        ),
        HumanMessage(content=input_message)
    ])
    return {"messages": [response]}

# Define the function that calls the model
def call_model_chat(state: State):
    """Provide a comprehensible response for a human"""
    messages = state['messages'][0].content
    response = model.invoke(messages)
    return {"messages": [response]}

# Conditional edge function to route to the appropriate node
def route_decision(state: State):
    if state["decision"] == "synthetizer":
        return "synthetizer"
    elif state["decision"] == "chat":
        return "chat"


# Define a new graph
workflow = StateGraph(State)
workflow.add_node("orchestrator", llm_call_router)
workflow.add_node("synthetizer", call_model_synthetize)
workflow.add_node("chat", call_model_chat)
workflow.add_node("tools", tool_node)
workflow.add_edge(START, "orchestrator")
workflow.add_conditional_edges(
    "orchestrator",
    route_decision, {
        "synthetizer": "synthetizer" ,
        "chat": "chat",
    },
)

# We now add a conditional edge
workflow.add_conditional_edges(
    "synthetizer",
    should_continue,
)

workflow.add_conditional_edges(
    "chat",
    should_continue,
)

workflow.add_conditional_edges(
    "tools",
    tools_callback,
)


app = workflow.compile()

final_state = app.invoke(
    {"messages": [HumanMessage(content="Blue lipstick below 10$")]},
    config={"configurable": {"thread_id": 42}}
)
print(final_state["decision"])
print(final_state["messages"][-1].content)

final_state = app.invoke(
    {"messages": [HumanMessage(content="hello!")]},
    config={"configurable": {"thread_id": 32}}
)
print(final_state["decision"])
print(final_state["messages"][-1].content)


synthetizer
- NYX Professional Makeup Suede Matte Lipstick - Blue
- Wet n Wild MegaLast Liquid Catsuit Lipstick - Blue
- e.l.f. Cosmetics Matte Lip Color - Blue
- LA Girl Matte Flat Finish Pigment Gloss - Blue
- Milani Color Statement Lipstick - Blue
- Rimmel London Lasting Finish Lipstick - Blue
- ColourPop Lippie Stix - Blue
- Maybelline Color Sensational Lip Color - Blue
chat
Hello! How can I assist you today?
